In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
from scipy.stats import norm, gmean, cauchy
import seaborn as sns
from datetime import datetime, timedelta

%matplotlib inline

In [5]:
# This code is from an online source, detailing how to use Cholesky decomposition in the BlackScholes processimport numpy as np
from scipy import stats
 
class BlackScholes:
    @staticmethod
    def Generate_Asset(S_0,R,T,Vol,X):
        return S_0*np.exp((R-Vol**2/2)*T + Vol*np.sqrt(T)*X)

class Monte_Carlo:
    @staticmethod
    def __Get_Correlated_Brownian(nb_assets,nb_simulation,correlation_matrix):
    #Function that returns a matrix with all the correlated brownian for all the simulations by proceeding a Cholesky decomposition"""
        X = np.random.randn(nb_simulation,nb_assets)
        lower_triang_cholesky = np.linalg.cholesky(correlation_matrix)
        for i in range(nb_simulation):
            X[i,:]=np.dot(lower_triang_cholesky,X[i,:])  #np.dot perform a matrix product
        return X
 
    @staticmethod
    def Get_Basket_Call_Price(starting_asset_values,correlation_matrix,asset_vol,maturity,nb_simulation,risk_free_rate,weights,strike):
        nb_assets = len(starting_asset_values)
 
        #Generate independant random variable:
        X = Monte_Carlo.__Get_Correlated_Brownian(nb_assets,nb_simulation,correlation_matrix)
 
        Final_Stock_values = BlackScholes.Generate_Asset(starting_asset_values[:],risk_free_rate,maturity,asset_vol[:],X[:])
 
        #print(Final_Stock_values[:])
        #print(weights)
        #print(Final_Stock_values[:]*weights)
        #print(np.sum(Final_Stock_values[:]*weights,axis=1))
        #print(np.maximum(np.sum(Final_Stock_values[:]*weights,axis=1)-strike,0))
 
        Payoffs = np.maximum(np.sum(Final_Stock_values[:]*weights,axis=1)-strike,0)
        return np.mean(Payoffs)*np.exp(-risk_free_rate*maturity)

class Margrabe:
    @staticmethod
    def __d1(S1,S2,T,sigma_eq):
        return (np.log(S1/S2)+0.5*T*sigma_eq**2)/(sigma_eq*np.sqrt(T))
 
    @staticmethod
    def __d2(S1,S2,T,sigma_eq):
        return Margrabe.__d1(S1,S2,T,sigma_eq)-sigma_eq*np.sqrt(T)
 
    @staticmethod
    def __sigma(vol1,vol2,correlation):
        return np.sqrt(vol1**2 + vol2**2 - 2*correlation*vol1*vol2)
 
    @staticmethod
    def Spread_Call_Price(S1,S2,vol1,vol2,T,correlation):
        sigma_eq = Margrabe.__sigma(vol1,vol2,correlation)
        d1 = Margrabe.__d1(S1,S2,T,sigma_eq)
        d2 = Margrabe.__d2(S1,S2,T,sigma_eq)
        return S1*stats.norm.cdf(d1)-S2*stats.norm.cdf(d2)

option_parameters_1 = {
    'starting_asset_values' : np.array([100,100]),
    'correlation_matrix':[[1,0.3],[0.3,1]],
    'asset_vol' : np.array([0.2,0.25]),
    'maturity' : 1,
    'nb_simulation' : 5000000,
    'risk_free_rate' : 0.01,
    'weights' : np.array([1,-1]),
    'strike' : 0
}
 
option_parameters_2 = {
    'starting_asset_values' : np.array([100,100,100]),
    'correlation_matrix':[[1,0.5,0.1],[0.5,1,0.7],[0.1,0.7,1]],
    'asset_vol' : np.array([0.2,0.25,0.22]),
    'maturity' : 1,
    'nb_simulation' : 5000000,
    'risk_free_rate' : 0.01,
    'weights' : np.array([0.4,0.2,0.4]),
    'strike' : 100
}
 
#Checking the result:
spread_option_price_MC = Monte_Carlo.Get_Basket_Call_Price(**option_parameters_1)
spread_option_price_margrabe = Margrabe.Spread_Call_Price(100,100,0.2,0.25,1,0.3)
 
#Example on a basket option with 3 stocks :
basket_option_price = Monte_Carlo.Get_Basket_Call_Price(**option_parameters_2)
 
print("Spread Option Price MC : {0}".format(spread_option_price_MC))
print("Spread Option Price Margrabe : {0}".format(spread_option_price_margrabe))
print("Basket Option Price on 3 stocks : {0}".format(basket_option_price))

Spread Option Price MC : 10.715123140813954
Spread Option Price Margrabe : 10.709488336629235
Basket Option Price on 3 stocks : 7.182811645972326
